# III Assignment - Routing Algorithms
## Student Massimo Palmisano
### Alghoritm to Add the shortcuts to a Graph

---
## Dependencies
Dependencies needed to run the following code

In [1]:
from ipynb.fs.full.exercise1 import relax, initialize_graph, min_distance
from heap import binheap
import copy

---
## Modified version of Dijkstra's Algorithm
In the contraction function we are interested in the minimum distance between two nodes. To get it we can use Dijkstra, but in a modified version, proposed in ```get_min_distance()```.

In [2]:
def get_min_distance_between_two_nodes(graph, source, destination):
    
    initialize_graph(graph, source)
    
    queue = binheap([[node, attributes.get('distance')] for node, attributes in graph.items()], total_order = min_distance)

    while not queue.is_empty():
        
        current_node, _ = queue.remove_minimum()
        graph.get(current_node)['visited'] = True    
        
        relax(graph, current_node, queue)

        if destination is current_node:
            break
            
    predecessors = build_predecessors(graph, destination)

    return graph.get(current_node).get('distance'), predecessors


---
## Predecessors
Build a tuple of nodes from source to destination following the shortest path. Hence, it takes a graph on which it has been already applied Dijkstra.

In [3]:
def build_predecessors(graph, destination):
    predecessors = list()
    current_node = destination
    while current_node is not None:
        predecessors.append(current_node)
        current_node = graph.get(current_node).get('predecessor')
    predecessors.reverse()
    return predecessors

---
## Contraction Function
Replace the node with shortcuts, where needed. 

In [4]:
def contract(graph, current_node):
    # searching all the nodes having a direct path to current_node...
    for node, attributes in graph.items():
        # ...for all these nodes we have to check the pair (node, neighbour_of_current_node)
        if current_node in attributes.get("adjacency_list").keys():
            for neighbour in graph.get(current_node)['adjacency_list'].keys():
                min_distance, predecessors = get_min_distance_between_two_nodes(copy.deepcopy(graph), node, neighbour)
                if current_node in predecessors:
                    graph.get(node).get('adjacency_list').setdefault(neighbour, min_distance)
            # delete the references to the deleted node
            attributes.get('adjacency_list').pop(current_node)
    graph.pop(current_node)   

---
## Algorithm to add the shortcuts to a graph
The alghorithm delete the nodes and arcs according to the Contraction Hierarchies Algorithm. In particular, in the following piece of code, the minimum importance parameter is given in order to delete all the nodes with an importance less than the one given

In [5]:
def contraction_hierarchies(graph, min_importance):
    for node, attributes in copy.deepcopy(graph).items():
        if attributes.get('importance') <= min_importance:
            contract(graph, node)
    return graph

---
## Example

In [10]:
if __name__ == '__main__':
    graph = {}
    graph.setdefault('A', {'adjacency_list': {'B': 3, 'E': 6}, 'importance': 1})
    graph.setdefault('B', {'adjacency_list': {'F': 1},         'importance': 2})
    graph.setdefault('C', {'adjacency_list': {'D': 3},         'importance': 2})
    graph.setdefault('D', {'adjacency_list': {},               'importance': 1})
    graph.setdefault('E', {'adjacency_list': {'D': 6},         'importance': 3})
    graph.setdefault('F', {'adjacency_list': {'C': 1},         'importance': 2})

    for node, attributes in contraction_hierarchies(graph, 0).items():
        print('Node', node, 'is survived with importance', attributes.get('importance'))


Node A is survived with importance 1
Node B is survived with importance 2
Node C is survived with importance 2
Node D is survived with importance 1
Node E is survived with importance 3
Node F is survived with importance 2
